# Google Natural Language API Client
## Setup
[https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-install-python](https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-install-python)

Run:
```
export GOOGLE_APPLICATION_CREDENTIALS="[PATH]"
```

## Sample Code
Run the below code to make sure you have everything set up. Something should show up similar to:
```
Text: Hello, world!
Sentiment: 0.300000011921, 0.300000011921
```

In [6]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: Hello, world!
Sentiment: 0.30000001192092896, 0.30000001192092896


# REST API Call

Export API Key to environmental variable:
```
export GOOGLE_KEY=MY_SECRET_KEY
```

In [7]:
import gzip

META_CELLPHONE = 'Datasets/meta_Cell_Phones_and_Accessories.json.gz'

def get_documents(file_name):
    g = gzip.open(file_name, 'r')
    results = []
    for line in g:
        document = eval(line)
        results.append(document)
    return results

In [8]:
all_documents = get_documents(META_CELLPHONE)

In [9]:
import os
api_key = os.environ['GOOGLE_KEY']

In [10]:
import requests
import json

LANGUAGE_ENDPOINT = 'https://language.googleapis.com/v1/documents:analyzeEntities?key={}'.format(api_key)

def merge_title_description(document):
    title = ''
    description = ''
    if 'title' in document:
        title = document['title']
    if 'description' in document:
        description = document['description']
    return title + " " + description

def perform_api_request(document):
    payload = {
        "document": {
            "type": "PLAIN_TEXT",
            "language": "EN",
            "content": merge_title_description(document)
        },
        "encodingType": "UTF8"
    }

    r = requests.post(LANGUAGE_ENDPOINT, data=json.dumps(payload))
    return r.text

In [11]:
def save_api_request(document_name, r):
    file_name = 'Processed/{}.json'.format(document_name)
    with open(file_name, 'w') as outfile:
        json_data = json.dump(r, outfile)

In [12]:
def serialize_processed_document(asin):
    already_processed = os.path.exists('Processed/{}.json'.format(asin))
    if already_processed:
        data = json.load(open('Processed/{}.json'.format(asin), 'r'))
        return json.loads(data)
    else:
        return None

In [13]:
import os

RUN_LIMIT = 5000
def process_all_documents(all_documents):
    count = 0
    for document in all_documents:
        asin = document['asin']
        already_processed = os.path.exists('Processed/{}.json'.format(asin))
        serialized = serialize_processed_document(asin)
        
        if not already_processed:
            r = perform_api_request(document)
            save_api_request(asin, r)
            count = count + 1
            if (count == RUN_LIMIT):
                break

process_all_documents(all_documents)

In [14]:
def read_files_into_memory(all_documents):
    serialized_documents = {}
    for document in all_documents:
        asin = document['asin']
        already_processed = os.path.exists('Processed/{}.json'.format(asin))
        if already_processed:
            serialized = serialize_processed_document(asin)
            serialized_documents[asin] = serialized
        else:
            return serialized_documents        

In [15]:
serialized = read_files_into_memory(all_documents)

In [16]:
def read_first_ten(serialized):
    count = 0
    for document in serialized:
        print(document)
        count += 1
        if count == 10:
            break
read_first_ten(serialized)

B001UOY2U4
B00374JHWS
B004WI3HSI
B004U5A7NG
B004Q20OHC
B00453824E
B002OKK4JQ
B004SG64X4
B000TVK94W
B003VPT6U6


## Substring matching to string match like CONSUMER_GOOD products

We will use this to classify and categorize different names of CONSUMER_GOOD, and see which ones match with one another.

First, we will remove capitilization and spaces. If we have time, we will need a classifier.

In [17]:
def resetText(text):
    text = ''.join(text.split()).lower()
    return text

## Highest Salience CONSUMER_GOOD

For now, we are just working with the highest salience CONSUMER_GOOD. Later, we will make it more complex and consider all CONSUMER_GOOD returned from each product. We will also be erasing duplicates that come up. The order of the items in the entities are in the order of highest salience.

In [18]:
def getConsumerGood(asin):
    # Function that takes in an asin number and returns the highest salience consumer good name
    if(asin in serialized):
        if('entities' in serialized[asin]):
            for items in serialized[asin]['entities']:
                if(items['type'] == "CONSUMER_GOOD"):
                    items['name'] = resetText(items['name'])
                    return items['name']
    return ""

In [19]:
all_categories = {}
def map_categories(all_documents):
    for document in all_documents:
        name = getConsumerGood(document['asin'])
        if(name == ""):
            continue
        for category in document['categories']:
            if(len(category) < 2):
                continue
            if (category[0] in all_categories):
                sub_category = all_categories[category[0]] # map<category, map<consumer_good, list<document> > >
                if(category[1] not in sub_category):
                    sub_category[category[1]] = {name: [document]}
                else:
                    if(name in sub_category[category[1]]):
                        sub_category[category[1]][name].append(document)
                    else:
                        sub_category[category[1]][name] = [document]
            else:
                all_categories[category[0]] = {category[1]: {name: [document]}}

In [20]:
map_categories(all_documents)

In [21]:
# Print all_categories 
def print_all_categories():
    i = 0
    nameList = []
    for category in all_categories:
        if (i == 10):
            break
        print('Category: {}'.format(category))
        for subcategory in all_categories[category]:
            print('\tSubcategory: {}'.format(subcategory))
            for name in all_categories[category][subcategory]:
                print('\t\tName: {}'.format(name))
                print('\t\t\tSize: {}'.format(len(all_categories[category][subcategory][name])))
        i = i + 1
print_all_categories()

Category: Automotive
	Subcategory: Tools & Equipment
		Name: carcharger
			Size: 1
		Name: adapter
			Size: 1
		Name: adapterthis
			Size: 1
	Subcategory: Replacement Parts
		Name: battery
			Size: 1
Category: Kindle Store
	Subcategory: Kindle Accessories
		Name: charger
			Size: 1
		Name: devicecompatible
			Size: 1
Category: Home & Kitchen
	Subcategory: Kitchen & Dining
		Name: chargingmat
			Size: 1
Category: Office Products
	Subcategory: Office Electronics
		Name: headsets
			Size: 1
		Name: headsetdect
			Size: 1
		Name: dect6.0
			Size: 1
		Name: pdaphone
			Size: 1
		Name: dialpad
			Size: 3
		Name: phone
			Size: 1
		Name: mobilehandset
			Size: 1
		Name: internetphone
			Size: 1
Category: Software
	Subcategory: Utilities
		Name: mobilephone
			Size: 1
		Name: phonedatatransfersuitesoftware
			Size: 1
		Name: software
			Size: 1
	Subcategory: Design & Illustration
		Name: cell-phone
			Size: 1
	Subcategory: Programming & Web Development
		Name: software
			Size: 1
	Subcategory:

		Name: lexuses300
			Size: 1
		Name: iphonebatteryboost
			Size: 1
		Name: blackif
			Size: 1
		Name: tworhinestonecherries
			Size: 1
		Name: htcmobilephones
			Size: 1
		Name: sinewaveinverter
			Size: 6
		Name: beltclipholster
			Size: 1
		Name: headsetearhookreplacementheadsetearhook
			Size: 1
		Name: 120gbpower
			Size: 1
		Name: instints30sprintthis
			Size: 1
		Name: sonyericssond750i
			Size: 1
		Name: lgchocolatevx8500vehicle
			Size: 1
		Name: tranceu490
			Size: 1
		Name: bluetoothheadsetinnovative
			Size: 1
		Name: juicepackair
			Size: 1
		Name: xv6900
			Size: 1
		Name: htc8925tilt
			Size: 2
		Name: 5325clreaderdesktop
			Size: 1
		Name: hermespda
			Size: 1
		Name: motorolah-500
			Size: 1
		Name: trackball
			Size: 7
		Name: charmnana
			Size: 1
		Name: fitzheadphoneadapter
			Size: 1
		Name: i776carchargercharge
			Size: 1
		Name: dellxps
			Size: 1
		Name: sprintnewcompatible
			Size: 1
		Name: printer
			Size: 1
		Name: 6670compatible
			Size: 1
		Name: simcardtr

		Name: retractablecable
			Size: 7
		Name: rapidchargetravelchargerfor2115i/2116i
			Size: 1
		Name: emergencyaabatterychargerextender
			Size: 5
		Name: samsunggalaxytab10.1vp7100
			Size: 1
		Name: htchd2mobile
			Size: 1
		Name: spn5532this
			Size: 1
		Name: homewalltravelchargercompatible
			Size: 1
		Name: blackberrybold
			Size: 9
		Name: originaliphone
			Size: 1
		Name: oemhigh-performanceli-ionbatteryforqv323v325snn5783motorolaoem
			Size: 1
		Name: lggw820
			Size: 1
		Name: fordflexindashmountmodel
			Size: 1
		Name: bluetoothheadsetuncluttered
			Size: 1
		Name: lexusis250
			Size: 1
		Name: windows
			Size: 2
		Name: headsetmusic
			Size: 1
		Name: a640a930originaloem
			Size: 1
		Name: sgh-a737
			Size: 1
		Name: microphones
			Size: 3
		Name: v750
			Size: 1
		Name: barragev860oem
			Size: 1
		Name: headsetheadphones
			Size: 35
		Name: sgh-a707a717a437a727need
			Size: 1
		Name: i1compatible
			Size: 1
		Name: samsunggalaxytab7.0
			Size: 5
		Name: ab553446gzbatterysc

			Size: 1
		Name: palmtreocentro
			Size: 1
		Name: travelhomechargerforsamsung&copy
			Size: 1
		Name: filtercompatible
			Size: 1
		Name: crystalsnap
			Size: 1
		Name: blackberrycurve8520
			Size: 7
		Name: packs
			Size: 1
		Name: fueltankadapter
			Size: 1
		Name: motoroladroid2globala956blueandwhitehawaiianflowersdesign
			Size: 1
		Name: htcozonexv6175
			Size: 2
		Name: device&apos
			Size: 2
		Name: rimplasticswivelholster
			Size: 1
		Name: mytouch3gslide
			Size: 10
		Name: chargepodmobilecharger
			Size: 1
		Name: fmstereoradioheadset
			Size: 1
		Name: htcnexusonedigitizer
			Size: 1
		Name: mobile
			Size: 11
		Name: htcone
			Size: 6
		Name: samsunginterceptm910
			Size: 4
		Name: cablespowercell
			Size: 1
		Name: slimline
			Size: 4
		Name: sonyericssonw518/w508
			Size: 1
		Name: datacable
			Size: 31
		Name: h605
			Size: 1
		Name: comet
			Size: 1
		Name: kc910renoir
			Size: 2
		Name: iphonetuffshield
			Size: 1
		Name: v180caselogicvehiclepowercharger
			Size: 1


			Size: 1
		Name: dreightonnebula
			Size: 2
		Name: blackberrytorch9800transparentclearcase
			Size: 1
		Name: parrotdriver
			Size: 1
		Name: nokia6300
			Size: 1
		Name: suitphone
			Size: 1
		Name: v7flasher
			Size: 1
		Name: truckercellularantennakit
			Size: 1
		Name: vuplusgr700
			Size: 1
		Name: nokialumia920cleartouchcrystal
			Size: 1
		Name: blackthis
			Size: 1
		Name: snn5704b
			Size: 1
		Name: blackberry8300
			Size: 1
		Name: desktopchargingdock
			Size: 1
		Name: lenovoideatab
			Size: 1
		Name: t229
			Size: 2
		Name: ipad3
			Size: 6
		Name: stereoplug
			Size: 1
		Name: htcevo4gclear
			Size: 1
		Name: beltclip&amp
			Size: 5
		Name: cdm8945
			Size: 1
		Name: headsetbh200
			Size: 1
		Name: mybatli-ionbattery
			Size: 1
		Name: bt-50
			Size: 1
		Name: iphone&#
			Size: 2
		Name: micro-usbtravelcharger
			Size: 1
		Name: motokrzr
			Size: 1
		Name: blackberrystyle9670this
			Size: 1
		Name: removableswivelbeltclipbelt
			Size: 1
		Name: sonywalkmane436e438mp3pla

...

			Size: 4
		Name: applemacbook
			Size: 1
		Name: dsct1t10t50
			Size: 1
		Name: motorolamilestonext720commuter
			Size: 1
		Name: hipholster5500
			Size: 1
		Name: iphone4-black-fitsat&amp
			Size: 22
		Name: cingular3125
			Size: 1
		Name: iphone4standcase
			Size: 1
		Name: a955droid2graphiccase
			Size: 4
		Name: hardplastic
			Size: 2
		Name: greensiliconeoverwhitehardcase
			Size: 1
		Name: blackberryswivelholster
			Size: 4
		Name: microshieldmix
			Size: 1
		Name: lgenvtouchhardcase
			Size: 1
		Name: solidblacksilicon
			Size: 2
		Name: abolisher
			Size: 2
		Name: microfiber
			Size: 2
		Name: a900
			Size: 3
		Name: thunder9500rubberizedprotectorhardcase
			Size: 1
		Name: blackberrytorch9800milkdrop
			Size: 1
		Name: rubbergelbumpercase
			Size: 1
		Name: globala955a956
			Size: 1
		Name: cabrio
			Size: 3
		Name: fireflymobilecellphonephone
			Size: 1
		Name: lx-160
			Size: 1
		Name: samsunggalaxysvibrantt959
			Size: 3
		Name: droida855.
			Size: 3
		Name: 9780thissili

			Size: 1
		Name: noble7nookwifi
			Size: 1
		Name: t669gravitytouch
			Size: 4
		Name: zugadgets
			Size: 2
		Name: designgoluggagedryphone
			Size: 1
		Name: cellphonenow
			Size: 3
		Name: bettyboopfilmstripdesign
			Size: 1
		Name: andrewextensis
			Size: 1
		Name: cellphoneaccessorieskit
			Size: 1
		Name: sportscars
			Size: 1
		Name: lgoctanevn530comborubberblackprotectivecase
			Size: 1
		Name: animalrubberbandbraceletthis
			Size: 1
		Name: krootmobile
			Size: 1
		Name: coverproduct
			Size: 2
		Name: butterflyflowercandy
			Size: 2
		Name: travelpackets
			Size: 2
		Name: motorolaxoomflexiskin
			Size: 1
		Name: beltloops
			Size: 1
		Name: shadesiphone
			Size: 5
		Name: motoroladroidproblackhardplasticcase
			Size: 1
		Name: env2vx-9100
			Size: 1
		Name: springflowerprotect
			Size: 1
		Name: syncsgh
			Size: 1
		Name: rubberizedface-in
			Size: 1
		Name: u640convoyturtlebackexecutivecase
			Size: 1
		Name: iphone3gkeeps
			Size: 1
		Name: authenticcelletheavydutyforceme

		Name: i-touch2
			Size: 1
		Name: purplefusion
			Size: 3
		Name: 32gb64gbthis
			Size: 4
		Name: scubacaseaccessoryone-nextel
			Size: 1
		Name: 3gsphilipsdlm2261
			Size: 1
		Name: headphonethis
			Size: 1
		Name: coverforsamsungi500
			Size: 2
		Name: iphone4geasy
			Size: 1
		Name: mb525
			Size: 8
		Name: motorolacliqxtdiamondbubblecase
			Size: 1
		Name: w845quantico
			Size: 1
		Name: productfit
			Size: 1
		Name: phonescelletnoblecaseseries
			Size: 1
		Name: tmobileblackcasepouch2ii3iiilxidsidekickii
			Size: 1
		Name: blackrubberizedprotectiveshield
			Size: 4
		Name: oden+blueswivelbeltclip
			Size: 1
		Name: redandwhiteflower
			Size: 4
		Name: gt-i9000
			Size: 2
		Name: scp-6780innuendographiccase-redflower
			Size: 1
		Name: production4gsmartphone
			Size: 8
		Name: blackberrybold9780
			Size: 1
		Name: foliopuleathercase
			Size: 1
		Name: w755
			Size: 3
		Name: elitelambskincase
			Size: 1
		Name: pantechp9050laser
			Size: 2
		Name: crsxxl
			Size: 1
		Name: speckp

			Size: 1
		Name: sonyericssont237
			Size: 1
		Name: eseries
			Size: 1
		Name: cingularattat&amp
			Size: 1
		Name: p1000
			Size: 1
		Name: musicplayer
			Size: 6
		Name: qtek9100
			Size: 1
		Name: symbianos
			Size: 4
		Name: thsrambler
			Size: 1
		Name: i90cphone
			Size: 1
		Name: samsungomniai910
			Size: 1
		Name: toolsthev505
			Size: 1
		Name: 6230i
			Size: 1
		Name: htcdroideris
			Size: 1
		Name: glydeglydesch-u940
			Size: 1
		Name: blackberry8350icurvefornextel
			Size: 1
		Name: sonyericssont226
			Size: 1
		Name: 3ggsmcellphone
			Size: 2
		Name: unlockednewpalmtreo680
			Size: 1
		Name: androidphone
			Size: 1
		Name: headset
			Size: 3
		Name: cellularphone
			Size: 23
		Name: smartphones
			Size: 1
		Name: cdmaphone
			Size: 4
		Name: 2g
			Size: 1
		Name: phonesonyericssonsonyericssons500
			Size: 1
		Name: multimediacomputer
			Size: 1
		Name: gx300
			Size: 1
		Name: moto
			Size: 1
		Name: windowsphone
			Size: 2
		Name: sim
			Size: 2
		Name: 2330c-2b
			Siz

			Size: 1
		Name: motorolaqwindowspdacdma
			Size: 1
		Name: quad-bandphone
			Size: 6
		Name: triband3g
			Size: 1
		Name: no-frillsphone
			Size: 1
		Name: palmpixiplus
			Size: 1
		Name: v180
			Size: 1
		Name: blackberrysmartphone
			Size: 1
		Name: multimediaphone
			Size: 1
		Name: theglofiishx500
			Size: 1
		Name: gs500cookieplus
			Size: 1
		Name: crystalclearhardcase
			Size: 1
		Name: ipaq910
			Size: 1
		Name: clamshellphone
			Size: 7
		Name: shuttlethevirginmobileshuttle
			Size: 1
		Name: laylometropcs
			Size: 1
		Name: s5230
			Size: 2
		Name: phonecandybar
			Size: 1
		Name: t401g
			Size: 1
		Name: i500
			Size: 2
		Name: m8910
			Size: 2
		Name: blackberrystorm2
			Size: 1
		Name: blackberrygemini
			Size: 1
		Name: blacklgcookielitet300
			Size: 1
		Name: crystalblue
			Size: 1
		Name: nokian72
			Size: 1
		Name: unlockedxplorer
			Size: 3
		Name: gsmworldcellphone
			Size: 1
		Name: cordelessphonepanasonic
			Size: 1
		Name: titaniumsony
			Size: 1
		Name: whiteb

## Query-based Algorithm
This will recommend three items from each sub-category that the product isn't classified as in the same overall category based on the query of the user. The query will come in as a document that has already been sent through Google Cloud Platform with results about the product.

In [84]:
def recommendItems(document):
    recommend = []
    title_description = document['title'] + " " + document['description']
    
    result = types.Document(content=title_description, type=enums.Document.Type.PLAIN_TEXT)
    entities = client.analyze_entities(document=result)
    for entity in entities.entities:
        if(entity.type == language.types.Entity.CONSUMER_GOOD):
            name = resetText(entity.name)
            for category in document['categories']:
                major_category = category[0]
                sub_category = category[1]
                if(major_category in all_categories):
                    for key in all_categories[major_category]:
                        if(key == sub_category):
                            continue
                        if(name not in all_categories[major_category][key]): #empty
                            continue
                        # Recommend all items from each category
                        for item in all_categories[major_category][key][name]:
                            recommend.append(item)
    return recommend

In [85]:
new_product = {'description': 'This is an iPhone3', 'title': 'iPhone3','categories': [['Cell Phones & Accessories', 'Cases', 'Basic Cases']],'asin': '1'}
recommended_items = recommendItems(new_product)

## Printing Recommended Items
Here is a function that will print items recommended.

In [88]:
def print_recommended_items(recommended_items):
    for item in recommended_items:
        print('Title: {}'.format(item['title']))
        if('description' in item):
            print('Description: {}'.format(item['description']))
        for category in item['categories']:
            print('Category: {}'.format(category[0]))
            print('\tSubcategory: {}'.format(category[1]))
        print('\n')

In [89]:
print_recommended_items(recommended_items)

Title: Hello Kitty Case with Screen Protector for iPhone 3/3G/3GS
Category: Cell Phones & Accessories
	Subcategory: Accessories


Title: RAM-B-149Z-PD3U: RAM Motorcycle and bicycle Universal Handlebar Mount for SmartPhone / iPhone 3/3GS/4 / HTC / Samsung / Motorola / LG Android Phone
Description: # Features
# MATERIAL: Powder Coated Marine Grade Aluminum
# BALL SIZE: 1 Inch Rubber Ball
# COLOR: Black Powder Coat
# WEIGHT: 0.68 lbs
# This cradle is designed to hold the following devices: Apple iPhone 3G, iPhone 3G S & iPhone 4
# Blackberry Bold & Storm 2
# HTC Droid Incredible, Evo 4G & G2
# Motorola Droid Pro, Droid 2 & Droid X
# Nokia N8, N900 & X6
# Samsung Galaxy S & Wave
# Windows 7 Smartphone
Category: Cell Phones & Accessories
	Subcategory: Accessories


Title: MiLi HI-C10 Power Pack for Iphone 3 (White/Orange)
Description: Works with iPhone in addition to Ipod Touch...
Certified by developers to meet Apple performance standards
Category: Cell Phones & Accessories
	Subcategory: A